# NPG-CMDP

---


今回はこちらの[論文](https://viterbi-web.usc.edu/~mihailo/papers/dinzhabasjovNIPS20.pdf)の紹介をしていきます。

まず、制約付きMDPの価値関数の定義をしていきます。

$r$は即時報酬、$g$は即時コストとします。

$$
V_r^\pi(s):=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid \pi, s_0=s\right] \text { , } V_g^\pi(s):=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t g\left(s_t, a_t\right) \mid \pi, s_0=s\right]
$$



そして、制約付きMDPでは従来の強化学習の価値最大化をもくひょうにしつつ、制約も満たすことを目標とします。

式で表すと、以下のようになります。

$$
\underset{\pi \in \Pi}{\operatorname{maximize}} V_r^\pi(\rho) \text { subject to } V_g^\pi(\rho) \geq b
$$



ラグランジュの未定乗数法で表すと、次のようになります。

$$
\underset{\theta \in \Theta}{\operatorname{maximize}} \underset{\lambda \geq 0}{\operatorname{minimize}} V_L^{\pi_\theta, \lambda}(\rho):=V_r^{\pi_\theta}(\rho)+\lambda\left(V_g^{\pi_\theta}(\rho)-b\right) \text {. }
$$



---


この論文の流れとしては、NPGの考えを使って、パラメータをそれぞれ更新していき、最後にモデルフリー(サンプルを使った)アルゴリズムを提案しています。

パラメータの更新則としては、次のようになります。

$$
\theta^{(t+1)}=\theta^{(t)}+\eta_1 F_\rho\left(\theta^{(t)}\right)^{\dagger} \cdot \nabla_\theta V_L^{\theta^{(t)}, \lambda^{(t)}}(\rho) \text { , } \lambda^{(t+1)}=\mathcal{P}_{\Lambda}\left(\lambda^{(t)}-\eta_2\left(V_g^{\theta^{(t)}}(\rho)-b\right)\right)
$$




まずは、上記の更新方法を使い、MDPが既知で、方策をソフトマックス関数で考えた時の、リグレットと、制約のviolationを以下に示します。

 $\frac{1}{T} \sum_{t=0}\left(V_r^*(\rho)-V_r^{(t)}(\rho)\right) \leq \frac{4}{(1-\gamma)^2} \frac{1}{\sqrt{T}}$ $\text{,}$ $\left[\frac{1}{T} \sum_{t=0}^{T-1}\left(b-V_g^{(t)}(\rho)\right)\right] \leq \frac{1 / \xi+4 \xi}{(1-\gamma)^2} \frac{1}{\sqrt{T}}$.

---

ここで、いくつかの記法を導入します。

まず、方策は$\pi_\theta(a \mid s)=\frac{\exp \left(\theta_{s, a}\right)}{\sum_{a^{\prime} \in A} \exp \left(\theta_{s, a^{\prime}}\right)}$、割引訪問確率は$d_{s_0}^\pi(s)=(1-\gamma) \sum_{t=0}^{\infty} \gamma^t P^\pi\left(s_t=s \mid s_0\right)$、フィッシャー情報行列は、$F_\rho(\theta):=\mathbb{E}_{s \sim d_\rho^{\pi_\theta}} \mathbb{E}_{a \sim \pi_\theta(\cdot \mid s)}\left[\nabla_\theta \log \pi_\theta(a \mid s)\left(\nabla_\theta \log \pi_\theta(a \mid s)\right)^{\top}\right]$とします。

次に、方策をsoftmax関数を使った時の、パラメータ更新を具体的に書いていきます。

パラメータ$\lambda$の更新範囲は$\Lambda=[0,2 /((1-\gamma) \xi)]$、報酬と制約を同時に考えた時のAdavantage関数$A_L^{(t)}(s, a):=A_r^{(t)}(s, a)+\lambda^{(t)} A_g^{(t)}(s, a)$、$Z^{(t)}(s):=\sum_{a \in A} \pi^{(t)}(a \mid s) \exp \left(\frac{\eta_1}{1-\gamma} A_L^{(t)}(s, a)\right)$.

このような記法を使うと、パラメータ、方策の更新は次のように考えられる。

$$
\theta_{s, a}^{(t+1)}=\theta_{s, a}^{(t)}+\frac{\eta_1}{1-\gamma} A_L^{(t)}(s, a)
$$

$$
\pi^{(t+1)}(a \mid s)=\pi^{(t)}(a \mid s) \frac{\exp \left(\frac{1}{1-\gamma} A_L^{(t)}(s, a)\right)}{Z^{(t)}(s)}
$$

$$
\lambda^{(t+1)}=\mathcal{P}_{\Lambda}\left(\lambda^{(t)}-\eta_2\left(V_g^{(t)}(\rho)-b\right)\right)
$$

---

証明をしていきましょう。

まずは、一回ごとの更新でmonotonic imporovement が発生していることを確認します。

$$
V_r^{(t+1)}(\mu)-V_r^{(t)}(\mu)+\lambda^{(t)}\left(V_g^{(t+1)}(\mu)-V_g^{(t)}(\mu)\right) \geq \frac{1-\gamma}{n_1} \mathbb{E}_{s \sim \mu} \log Z^{(t)}(s)
$$

$Z^t$は正なので、成り立ってそうですね。

**証明**

$$
\begin{aligned}
V_r^{(t+1)}(\mu)-V_r^{(t)}(\mu)= & \frac{1}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{(t+1)}, a \sim \pi^{(t+1)}(\cdot \mid s)}\left[A_r^{(t)}(s, a)\right] \\
= & \frac{1}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[\sum_{a \in A} \pi^{(t+1)}(a \mid s) A_r^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[\sum_{a \in A} \pi^{(t+1)}(a \mid s) \log \left(\frac{\pi^{(t+1)}(a \mid s)}{\pi^{(t)}(a \mid s)} Z^{(t)}(s)\right)\right] \\
& -\frac{\lambda^{(t)}}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[\sum_{a \in A} \pi^{(t+1)}(a \mid s) A_g^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[D_{\mathrm{KL}}\left(\pi^{(t+1)}(a \mid s) \| \pi^{(t)}(a \mid s)\right)\right] \\
& +\frac{1}{\eta_1} \mathbb{E}_{s \sim d_\mu^{(t+1)}} \log Z^{(t)}(s) \\
& -\frac{\lambda^{(t)}}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[\sum_{a \in A} \pi^{(t+1)}(a \mid s) A_g^{(t)}(s, a)\right] \\
\geq & \frac{1}{\eta_1} \mathbb{E}_{s \sim d_\mu^{(t+1)}} \log Z^{(t)}(s) \\
& -\frac{\lambda^{(t)}}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{(t+1)}}\left[\sum_{a \in A} \pi^{(t+1)}(a \mid s) A_g^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d_\mu^{(t+1)}} \log Z^{(t)}(s)-\lambda^{(t)}\left(V_g^{(t+1)}(\mu)-V_g^{(t)}(\mu)\right)
\end{aligned}
$$

次に、T回の試行での最適な価値関数との平均誤差をしらべます。

証明には先ほど示した、monotonic improvementを終盤に使います。

$$
\frac{1}{T} \sum_{t=0}^{T-1}\left(V_r^{\star}(\rho)-V_r^{(t)}(\rho)\right)+\frac{1}{T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right) \leq \frac{\log |A|}{\eta_1 T}+\frac{1}{(1-\gamma)^2 T}+\frac{2 \eta_2}{(1-\gamma)^3}
$$

**証明**

$$
\begin{aligned}
V_r^{\star}(\rho)-V_r^{(t)}(\rho)= & \frac{1}{1-\gamma} \mathbb{E}_{s \sim d^{\star}}\left[\sum_{a \in A} \pi^{\star}(a \mid s) A_r^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d^{\star}}\left[\sum_{a \in A} \pi^{\star}(a \mid s) \log \left(\frac{\pi^{(t+1)}(a \mid s)}{\pi^{(t)}(a \mid s)} Z^{(t)}(s)\right)\right] \\
& -\frac{\lambda^{(t)}}{1-\gamma} \mathbb{E}_{s \sim d^{\star}}\left[\sum_{a \in A} \pi^{\star}(a \mid s) A_g^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d^{\star}}\left[D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t)}(a \mid s)\right)-D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t+1)}(a \mid s)\right)\right] \\
& +\frac{1}{\eta_1} \mathbb{E}_{s \sim d^{\star}} \log Z^{(t)}(s)-\frac{\lambda^{(t)}}{1-\gamma} \mathbb{E}_{s \sim d^{\star}}\left[\sum_{a \in A} \pi^{\star}(a \mid s) A_g^{(t)}(s, a)\right] \\
= & \frac{1}{\eta_1} \mathbb{E}_{s \sim d^{\star}}\left[D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t)}(a \mid s)\right)-D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t+1)}(a \mid s)\right)\right] \\
& +\frac{1}{\eta_1} \mathbb{E}_{s \sim d^{\star}} \log Z^{(t)}(s)-\lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right)
\end{aligned}
$$

ここで、$Z^{(t)}$を先ほどのmonotonic improvement で得た結果を使って書き直したいと思います。

$$
V_r^{(t+1)}\left(d^{\star}\right)-V_r^{(t)}\left(d^{\star}\right)+\lambda^{(t)}\left(V_g^{(t+1)}\left(d^{\star}\right)-V_g^{(t)}\left(d^{\star}\right)\right) \geq \frac{1-\gamma}{\eta_1} \mathbb{E}_{s \sim d^{\star}} \log Z^{(t)}(s)
$$



上の変換を使い、平均誤差を求める。

$$
\begin{aligned}
& \frac{1}{T} \sum_{t=0}^{T-1}\left(V_r^{\star}(\rho)-V_r^{(t)}(\rho)\right) \\
& =\frac{1}{\eta_1 T} \sum_{t=0}^{T-1} \mathbb{E}_{s \sim d^{\star}}\left[D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t)}(a \mid s)\right)-D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t+1)}(a \mid s)\right)\right] \\
& +\frac{1}{\eta_1 T} \sum_{t=0}^{T-1} \mathbb{E}_{s \sim d^{\star}} \log Z^{(t)}(s)-\frac{1}{T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right) \\
& \leq \frac{1}{\eta_1 T} \sum_{t=0}^{T-1} \mathbb{E}_{s \sim d^{\star}}\left[D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t)}(a \mid s)\right)-D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(t+1)}(a \mid s)\right)\right] \\
& +\frac{1}{(1-\gamma) T} \sum_{t=0}^{T-1}\left(V_r^{(t+1)}\left(d^{\star}\right)-V_r^{(t)}\left(d^{\star}\right)\right) \\
& +\frac{1}{(1-\gamma) T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{(t+1)}\left(d^{\star}\right)-V_g^{(t)}\left(d^{\star}\right)\right)-\frac{1}{T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right) \\
& \leq \frac{1}{\eta_1 T} \mathbb{E}_{s \sim d^{\star}} D_{\mathrm{KL}}\left(\pi^{\star}(a \mid s) \| \pi^{(0)}(a \mid s)\right)+\frac{1}{(1-\gamma) T} V_r^{(T)}\left(d^{\star}\right)+\frac{2 \eta_2}{(1-\gamma)^3} \\
& -\frac{1}{T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right) \\
&
\end{aligned}
$$

$D_{\mathrm{KL}}(p \| q) \leq \log |A|$と、$V_r^{(T)}\left(d^{\star}\right) \leq \frac{1}{1-\gamma}$を使うと、下のようにまとめられる。

$$
\begin{aligned}
& \frac{1}{T} \sum_{t=0}^{T-1}\left(V_r^{\star}(\rho)-V_r^{(t)}(\rho)\right)+\frac{1}{T} \sum_{t=0}^{T-1} \lambda^{(t)}\left(V_g^{\star}(\rho)-V_g^{(t)}(\rho)\right) \\
& \leq \frac{\log |A|}{\eta_1 T}+\frac{1}{(1-\gamma)^2 T}+\frac{2 \eta_2}{(1-\gamma)^3}
\end{aligned}
$$



$\frac{1}{T} \sum_{t=0}^{T-1}\left(V_r^{\star}(\rho)-V_r^{(t)}(\rho)\right)$の部分と、対応している項は$\frac{\log |A|}{\eta_1 T}+\frac{1}{(1-\gamma)^2 T}$この項なので、右辺が最小となる$\eta_1$を求めてあげればいい。

そのようにして、求めると、$\eta_1=2 \log |A|$になる。

**この求め方がいまいちわからない。相加相乗平均の知識を使ってるのかな？**